In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
import torch
from matplotlib import pyplot as plt
import pickle

In [20]:
class CBOW(torch.nn.Module):
    def __init__(self,vocab_size,embedding_dim,context_size):
        super(CBOW,self).__init__()
        self.embeddings = torch.nn.Embedding(vocab_size,embedding_dim)
        self.linear1 = torch.nn.Linear(context_size*embedding_dim,128)
        self.linear2 = torch.nn.Linear(128,vocab_size)

    def forward(self,inputs):
        embeds = self.embeddings(inputs).view(1,-1)
        out = torch.nn.functional.relu(self.linear1(embeds))
        out = self.linear2(out)
        return out

In [214]:
def search_close_word(word):
    # id = word_to_ix[word]
    # #print(id)
    # w1 = w[id]
    # cos_sim = []
    # for key in word_to_ix:
    #     id2 = word_to_ix[key]
    #     w2 = w[id2]
    #     sim = cos(w1,w2).item()
    #     cos_sim.append((sim,key))
    # cos_sim.sort(key=lambda x:-x[0])
    if(word in trained_model):
        word1 = torch.from_numpy(trained_model[word].astype(np.float32)).clone()
    cos_sim = []
    for key in word_to_ix:
        if(key in trained_model):
            #print(key)
            word2 = torch.from_numpy(trained_model[key].astype(np.float32)).clone()
            sim = cos(word1,word2)
            cos_sim.append((sim,key))
    cos_sim.sort(key=lambda x:-x[0])
    return cos_sim


In [281]:
path = './output/vocab2.pkl'
word_to_ix = pickle.load(open(path,'rb'))
# print(word_to_ix)
# print(word_to_ix['him'])
model_path = './models2/model_399'
vocab_size = len(word_to_ix)
#print(word_to_ix)


EMBEDDING_DIM  = 16
CONTEXT_SIZE = 2


model = CBOW(vocab_size,EMBEDDING_DIM,CONTEXT_SIZE)
model.load_state_dict(torch.load(model_path))
w = model.state_dict()['embeddings.weight']
print(w.size())
cos = torch.nn.CosineSimilarity(dim=-1,eps=1e-6)
# vec1 = torch.from_numpy(trained_model['when'].astype(np.float32)).clone()
# vec2 = torch.from_numpy(trained_model['when'].astype(np.float32)).clone()
#print(cos(vec1,vec2))
# cos_sim = search_close_word('him')

# print(cos_sim)
#print(word_to_ix)

torch.Size([1945, 16])


In [282]:
input_path = '../test_data/STS.input.images.txt'
output_path = './output/STS.output.images.txt'
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
#ファイル入力
def file_input(input_path):
    with open(input_path,encoding='UTF-8-sig') as f:
        s = f.read()
    s = s.strip().replace('\t','\n').replace('.','')
    sentences = s.split('\n')
    ps = PorterStemmer()
    for i in range(len(sentences)):
        token = sentences[i].split(' ')
        singles = [ps.stem(plural) for plural in token]
        #sentences[i] = token
        sentences[i] = singles
    return sentences

def sub_char(sentences:list):
    new_sentenes = []
    for sentence in sentences:
        new_sentence = []
        for word in sentence:
            w = word.replace(':','').replace(';','').replace('(','').replace(')','').replace(',','').replace('.','')
            new_sentence.append(w)
        new_sentenes.append(new_sentence)
    return new_sentenes

sentences = file_input(input_path)
sentences = sub_char(sentences)
print(sentences)

#Vectorizerを設定する。
l = len(sentences)
print(l)

#c750個のcos類似度リスト
cos_sim_list = []
#出力用の文字列
output = ''

# vecs1 = []
# vecs2 = []

# print(w[word_to_ix['him']])

#それぞれ類似度を求める。
for i in range(int(l/2)):

    vec1 = torch.zeros(16)
    vec2 = torch.zeros(16)
    for word in sentences[2*i]:
        if (word in word_to_ix):
            vec1 += w[word_to_ix[word]]
    for word in sentences[2*1+1]:
        if (word in word_to_ix):
            vec2 += w[word_to_ix[word]]
    print(i)
    value = cos(vec1,vec2).item() + 1
    print(value)
    output += str(value) + '\n'
# cos_sim = cosine_similarity(vecs1,vecs2)
with open(output_path,mode='w') as f:
    f.write(output)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fox25\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['a', 'cat', 'stand', 'on', 'tree', 'branch'], ['a', 'black', 'and', 'white', 'cat', 'is', 'high', 'up', 'on', 'tree', 'branch'], ['two', 'green', 'and', 'white', 'train', 'sit', 'on', 'the', 'track'], ['two', 'green', 'and', 'white', 'train', 'on', 'track'], ['a', 'small', 'white', 'cat', 'with', 'glow', 'eye', 'stand', 'underneath', 'a', 'chair'], ['a', 'white', 'cat', 'stand', 'on', 'the', 'floor'], ['a', 'larg', 'boat', 'in', 'the', 'water', 'at', 'the', 'marina'], ['a', 'larg', 'boat', 'on', 'the', 'sea'], ['a', 'bu', 'drive', 'in', 'a', 'street'], ['red', 'doubl', 'decker', 'bu', 'drive', 'down', 'street'], ['the', 'lamb', 'is', 'look', 'at', 'the', 'camera'], ['a', 'small', 'bird', 'stand', 'on', 'a', 'log', 'at', 'the', 'water', 'edg'], ['a', 'passeng', 'train', 'wait', 'in', 'a', 'station'], ['a', 'passeng', 'train', 'sit', 'in', 'the', 'station'], ['a', 'woman', 'at', 'a', 'dinner', 'tabl', 'write', 'on', 'her', 'notebook'], ['woman', 'at', 'tabl', 'busi', 'with', 'someth'],

In [180]:
from gensim.models import KeyedVectors

trained_model = KeyedVectors.load_word2vec_format('cc.en.300.vec.gz',binary=False)

In [224]:
a  = trained_model['hello']
x = torch.from_numpy(a.astype(np.float32)).clone()
print(x.size())
y = torch.zeros(300)
print(y.size())

torch.Size([300])
torch.Size([300])


In [225]:
#それぞれ類似度を求める。
count = 0
output = ''
for i in range(int(l/2)):
    for word in sentences[2*i]:
        sec1 = torch.zeros(300)
        sec2 = torch.zeros(300)
        if(word in trained_model):
            word1 = torch.from_numpy(trained_model[word].astype(np.float32)).clone()
            sec1 += word1
        else:
            print('aaaaaaaaaa')
            print(count)
            count += 1
    for word in sentences[2*1+1]:
        if(word in trained_model):
            word2 = torch.from_numpy(trained_model[word].astype(np.float32)).clone()
            sec2 += word2
        else:
            print('bbbbbbbbbbbbbbbbb')
            print(count)
            count += 1
    value = cos(sec1,sec2).item() 
    output += str(value) + '\n'
    #print(value)
# cos_sim = cosine_similarity(vecs1,vecs2)
print(output)
with open(output_path,mode='w') as f:
    f.write(output)

aaaaaaaaaa
0
aaaaaaaaaa
1
aaaaaaaaaa
2
aaaaaaaaaa
3
aaaaaaaaaa
4
aaaaaaaaaa
5
aaaaaaaaaa
6
aaaaaaaaaa
7
aaaaaaaaaa
8
aaaaaaaaaa
9
aaaaaaaaaa
10
aaaaaaaaaa
11
aaaaaaaaaa
12
aaaaaaaaaa
13
aaaaaaaaaa
14
aaaaaaaaaa
15
aaaaaaaaaa
16
aaaaaaaaaa
17
aaaaaaaaaa
18
aaaaaaaaaa
19
aaaaaaaaaa
20
aaaaaaaaaa
21
aaaaaaaaaa
22
aaaaaaaaaa
23
aaaaaaaaaa
24
aaaaaaaaaa
25
aaaaaaaaaa
26
aaaaaaaaaa
27
aaaaaaaaaa
28
aaaaaaaaaa
29
aaaaaaaaaa
30
aaaaaaaaaa
31
aaaaaaaaaa
32
aaaaaaaaaa
33
aaaaaaaaaa
34
aaaaaaaaaa
35
aaaaaaaaaa
36
aaaaaaaaaa
37
aaaaaaaaaa
38
aaaaaaaaaa
39
aaaaaaaaaa
40
aaaaaaaaaa
41
aaaaaaaaaa
42
aaaaaaaaaa
43
aaaaaaaaaa
44
aaaaaaaaaa
45
aaaaaaaaaa
46
aaaaaaaaaa
47
aaaaaaaaaa
48
aaaaaaaaaa
49
aaaaaaaaaa
50
aaaaaaaaaa
51
aaaaaaaaaa
52
aaaaaaaaaa
53
aaaaaaaaaa
54
aaaaaaaaaa
55
aaaaaaaaaa
56
aaaaaaaaaa
57
aaaaaaaaaa
58
aaaaaaaaaa
59
aaaaaaaaaa
60
aaaaaaaaaa
61
aaaaaaaaaa
62
aaaaaaaaaa
63
aaaaaaaaaa
64
aaaaaaaaaa
65
aaaaaaaaaa
66
aaaaaaaaaa
67
aaaaaaaaaa
68
aaaaaaaaaa
69
aaaaaaaaaa
70
aaaaaaaaaa
71
aa

In [280]:
count = 0
output = ''
from scipy import spatial
import numpy as np
# def avg_feature_vector(sentence):
#     feature_vec = np.zeros(300,dtype='float32')
#     for word in sentence:
#         if word in trained_model:
#             #print(word,trained_model[word])
#             feature_vec = np.add(feature_vec,trained_model[word])
#             #print(feature_vec)
#     #feature_vec = np.divide(feature_vec,len(sentence))
#     return feature_vec

def avg_feature_vector(sentence):
    feature_vec = np.zeros((300,), dtype="float32") # 特徴ベクトルの入れ物を初期化
    for word in sentence:
        if word in trained_model and not word in stopwords:
            feature_vec = np.add(feature_vec, trained_model[word])
    if len(sentences) > 0:
        feature_vec = np.divide(feature_vec, len(sentence))
    return feature_vec
a = avg_feature_vector(sentences[0])
print(a)    
def sentence_similarity(sentence1,sentence2):
    sentence1_vec = avg_feature_vector(sentence1)
    sentenve2_vec = avg_feature_vector(sentence2)
    #print(sentence1_vec,sentenve2_vec)
    return (1 - spatial.distance.cosine(sentence1_vec,sentenve2_vec))
c = sentence_similarity(sentences[0],sentences[1])
print(c)

l = len(sentences)
    
for i in range(int(l/2)):
    value = sentence_similarity(sentences[2*i],sentences[2*i+1])
    output += str(value) + '\n'
    print(value)
# cos_sim = cosine_similarity(vecs1,vecs2)
print(output)
with open(output_path,mode='w') as f:
    f.write(output)

[ 0.00245    -0.00918333  0.00608333  0.09895     0.00125    -0.02616666
  0.0673      0.02228334 -0.01466667  0.0266     -0.04056667 -0.04816667
 -0.0063     -0.03831667  0.03431666  0.06648333  0.03671667 -0.00983333
 -0.00633333  0.00631667  0.01473333 -0.01626667  0.0461     -0.01426667
 -0.01365    -0.10013333  0.06171666 -0.01776666 -0.0029      0.0886
 -0.0369      0.01863333  0.01946666 -0.01715    -0.02413333 -0.0576
 -0.03281667  0.03038334 -0.0328     -0.06254999 -0.02495     0.03031667
 -0.00361667  0.02176667 -0.07643333  0.03058334  0.04405    -0.01901667
 -0.03268333  0.00598333 -0.02585    -0.04106667  0.0246     -0.0233
 -0.08488333 -0.03818334 -0.00235    -0.01325    -0.05828333  0.01783333
  0.0721      0.07621667  0.01678333  0.02026667 -0.02885     0.00573333
 -0.008       0.03578334 -0.02991667 -0.00408333 -0.01008333 -0.03035
  0.00963333 -0.0189     -0.03718333 -0.02523333  0.04268333 -0.01101667
 -0.02373333 -0.00666667  0.02958333 -0.0121     -0.04055    -0.00

In [271]:
print(sentences[10],sentences[11])
print(sentence_similarity(sentences[10],sentences[11]))

['the', 'lamb', 'is', 'look', 'at', 'the', 'camera'] ['a', 'small', 'bird', 'stand', 'on', 'a', 'log', 'at', 'the', 'water', 'edg']
0.811175525188446
